In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, silhouette_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
desired_columns = ["MainBranch",
"Employment",
"RemoteWork",
"CodingActivities",
"EdLevel",
"LearnCode",
"LearnCodeOnline",
"LearnCodeCoursesCert",
"YearsCode",
"YearsCodePro",
"DevType",
"Country",
"LanguageHaveWorkedWith",
"LanguageWantToWorkWith",
"DatabaseHaveWorkedWith",
"DatabaseWantToWorkWith",
"PlatformHaveWorkedWith",
"PlatformWantToWorkWith",
"WebframeHaveWorkedWith",
"WebframeWantToWorkWith",
"MiscTechHaveWorkedWith",
"MiscTechWantToWorkWith",
"ToolsTechHaveWorkedWith",
"ToolsTechWantToWorkWith",
"NEWCollabToolsHaveWorkedWith",
"NEWCollabToolsWantToWorkWith",
"ICorPM",
"WorkExp",
"ProfessionalTech",
"Industry"]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
originalDF =  pd.read_csv('/content/drive/MyDrive/Eduforge/survey_results_public.csv')

In [ ]:
originalDF = originalDF[desired_columns]

In [ ]:
originalDF.shape

(89184, 30)

In [ ]:
numerical_columns = ['YearsCode', 'YearsCodePro', 'WorkExp']
categorical_columns = originalDF.columns.difference(numerical_columns)
categorical_columns

Index(['CodingActivities', 'Country', 'DatabaseHaveWorkedWith',
       'DatabaseWantToWorkWith', 'DevType', 'EdLevel', 'Employment', 'ICorPM',
       'Industry', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
       'LearnCode', 'LearnCodeCoursesCert', 'LearnCodeOnline', 'MainBranch',
       'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith',
       'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith',
       'PlatformHaveWorkedWith', 'PlatformWantToWorkWith', 'ProfessionalTech',
       'RemoteWork', 'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
       'WebframeHaveWorkedWith', 'WebframeWantToWorkWith'],
      dtype='object')

In [ ]:
originalDF.head()

,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,YearsCodePro,...,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,ICorPM,WorkExp,ProfessionalTech,Industry
0,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I am a developer by profession,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Bootstrapping a business;Professional development or self-paced learning from online courses,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;Colleague;Friend or family member;Hackathons (virtual or in-person);Online Courses or Certification;On the job training;Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc)","Formal documentation provided by the owner of the tech;Blogs with tips and tricks;Books;Recorded coding sessions;How-to videos;Video-based Online Courses;Written-based Online Courses;Auditory material (e.g., podcasts);Online challenges (e.g., daily or weekly coding challenges);Written Tutorials;Click to write Choice 20;Stack Overflow",Other,18,9,...,Electron;React Native;Tauri,Capacitor;Electron;Tauri;Uno Platform;Xamarin,Docker;Kubernetes;npm;Pip;Vite;Webpack;Yarn,Godot;npm;pnpm;Unity 3D;Unreal Engine;Vite;Webpack;Yarn,Vim;Visual Studio Code,Vim;Visual Studio Code,People manager,10.0,DevOps function;Microservices;Automated testing;Observability tools,"Information Services, IT, Software Development, or other Technology"
2,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced learning from online courses,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;Colleague;On the job training;Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc)","Formal documentation provided by the owner of the tech;Blogs with tips and tricks;How-to videos;Online challenges (e.g., daily or weekly coding challenges);Written Tutorials;Click to write Choice 20;Stack Overflow",NaN,27,23,...,NaN,NaN,Cargo;Docker;Kubernetes;Make;Nix,Cargo;Kubernetes;Nix,Emacs;Helix,Emacs;Helix,Individual contributor,23.0,DevOps function;Microservices;Automated testing;Observability tools;Innersource initiative;Developer portal or other central places to find tools/services;Continuous integration (CI) and (more often) continuous delivery,"Information Services, IT, Software Development, or other Technology"
3,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Colleague;Friend or family member;Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc)","Formal documentation provided by the owner of the tech;Blogs with tips and tricks;Auditory material (e.g., podcasts);Written Tutorials;Stack Overflow;Interactive tutorial",NaN,12,7,...,NaN,NaN,Homebrew;npm;Vite;Webpack;Yarn,Homebrew;npm;Vite,IntelliJ IDEA;Vim;Visual Studio Code;WebStorm,IntelliJ IDEA;Vim;WebStorm,Individual contributor,7.0,Automated testing;Continuous integration (CI) and (more often) continuous delivery,NaN
4,I am a developer by profession,"Employed, full-time;Independent contractor, freelancer, or self-employed",Remote,Hobby;Contribute to open-source projects;Professional development or self-paced learning from online courses,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;Online Courses or Certification;Other online resources (e.g., videos, blogs, forum);School (i.e., University, College, etc)","Formal documentation provided by the owner of the tech;Blogs with tips and tricks;Books;How-to videos;Video-based Online Courses;Online challenges (e.g., daily or weekly coding challenges);Written Tutorials;Click to write Choice 20;Interactive t

In [ ]:
df = originalDF.fillna("NA")

In [ ]:
######## NO ###############
# Get unique values for each column
unique_values_per_column = {}

for column in df.columns:
    unique_values_per_column[column] = df[column].unique()

# Print unique values for each column
for column, unique_values in unique_values_per_column.items():
    print(f"Unique values for column '{column}':")
    print(unique_values)
    print()

In [ ]:
#One-Hot Encoding
from sklearn.preprocessing import MultiLabelBinarizer
# Identify columns with values separated by semi-colon
semicolon_cols = [col for col in categorical_columns if any(';' in value for value in df[col])]

# Apply one-hot encoding to identified columns
for col in semicolon_cols:
    # Split values into separate columns
    df[col] = df[col].str.split(';')

    # Initialize MultiLabelBinarizer
    mlb = MultiLabelBinarizer()

    # Set custom column names
    mlb.fit(df[col])
    mlb.classes_ = [f'{col}_{c}' for c in mlb.classes_]

    # Apply one-hot encoding
    encoded_categories = pd.DataFrame(mlb.transform(df[col]), columns=mlb.classes_, index=df.index)

    # Concatenate the encoded columns with the original DataFrame
    df = pd.concat([df, encoded_categories], axis=1)

    # Drop the original column
    df.drop(col, axis=1, inplace=True)

print(df.columns)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Bootstrapping a business', 'Contribute to open-source projects', 'Freelance/contract work', 'Hobby', 'I don’t code outside of work', 'NA', 'Other (please specify):', 'Professional development or self-paced learning from online courses', 'School or academic work'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['BigQuery', 'Cassandra', 'Clickhouse', 'Cloud Firestore', 'Cockroachdb', 'Cosmos DB', 'Couch DB', 'Couchbase', 'Datomic', 'DuckDB', 'Dynamodb', 'Elasticsearch', 'Firebase Realtime Database', 'Firebird', 'H2', 'IBM DB2', 'InfluxDB', 'MariaDB', 'Microsoft Access', 'Microsoft SQL Server', 'MongoDB', 'MySQL', 'NA', 'Neo4J', 'Oracle', 'PostgreSQL', 'RavenDB', 'Redis', 'SQLite', 'Snowflake', 'Solr', 'Supabase', 'TiDB'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-pac

Index(['MainBranch', 'RemoteWork', 'EdLevel', 'YearsCode', 'YearsCodePro',
       'DevType', 'Country', 'ICorPM', 'WorkExp', 'Industry',
       ...
       'WebframeWantToWorkWith_React', 'WebframeWantToWorkWith_Remix',
       'WebframeWantToWorkWith_Ruby on Rails',
       'WebframeWantToWorkWith_Solid.js', 'WebframeWantToWorkWith_Spring Boot',
       'WebframeWantToWorkWith_Svelte', 'WebframeWantToWorkWith_Symfony',
       'WebframeWantToWorkWith_Vue.js', 'WebframeWantToWorkWith_WordPress',
       'WebframeWantToWorkWith_jQuery'],
      dtype='object', length=630)


In [ ]:
remaining_categorical_columns = list(set(categorical_columns) - set(semicolon_cols))
remaining_categorical_columns

['Country',
 'EdLevel',
 'RemoteWork',
 'MainBranch',
 'Industry',
 'ICorPM',
 'DevType']

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder

# # Create an instance of One-hot-encoder
# enc = OneHotEncoder()

# enc_data = pd.DataFrame(enc.fit_transform(
#     df[remaining_categorical_columns]).toarray())


# df = df.join(enc_data)


df = pd.get_dummies(df, columns = remaining_categorical_columns,drop_first=True)

In [ ]:
print(df.columns.tolist())

['YearsCode', 'YearsCodePro', 'WorkExp', 'CodingActivities_Bootstrapping a business', 'CodingActivities_Contribute to open-source projects', 'CodingActivities_Freelance/contract work', 'CodingActivities_Hobby', 'CodingActivities_I don’t code outside of work', 'CodingActivities_NA', 'CodingActivities_Other (please specify):', 'CodingActivities_Professional development or self-paced learning from online courses', 'CodingActivities_School or academic work', 'DatabaseHaveWorkedWith_BigQuery', 'DatabaseHaveWorkedWith_Cassandra', 'DatabaseHaveWorkedWith_Clickhouse', 'DatabaseHaveWorkedWith_Cloud Firestore', 'DatabaseHaveWorkedWith_Cockroachdb', 'DatabaseHaveWorkedWith_Cosmos DB', 'DatabaseHaveWorkedWith_Couch DB', 'DatabaseHaveWorkedWith_Couchbase', 'DatabaseHaveWorkedWith_Datomic', 'DatabaseHaveWorkedWith_DuckDB', 'DatabaseHaveWorkedWith_Dynamodb', 'DatabaseHaveWorkedWith_Elasticsearch', 'DatabaseHaveWorkedWith_Firebase Realtime Database', 'DatabaseHaveWorkedWith_Firebird', 'DatabaseHaveWor

In [ ]:
df['Country_Congo, Republic of the... ']

In [ ]:
######## NO ###############
# Encode categorical variables
label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

In [ ]:
df['YearsCode'].unique()
df['YearsCode'] = df['YearsCode'].replace('Less than 1 year',1)
df['YearsCode'] = df['YearsCode'].replace( 'More than 50 years',51)
df['YearsCode'] = df['YearsCode'].replace('NA',0)
df['YearsCode'].unique()

array([0, '18', '27', '12', '6', '21', '4', '5', '20', '14', '10', '15',
       '11', '3', '24', '8', '13', 1, '16', '33', '22', '30', '32', '7',
       '35', '28', '40', '17', '29', '19', 51, '9', '38', '26', '34',
       '25', '2', '45', '23', '31', '43', '1', '48', '41', '50', '39',
       '42', '37', '36', '44', '46', '49', '47'], dtype=object)

In [ ]:
df['YearsCodePro'].unique()
df['YearsCodePro'] = df['YearsCodePro'].replace('Less than 1 year',1)
df['YearsCodePro'] = df['YearsCodePro'].replace( 'More than 50 years',51)
df['YearsCodePro'] = df['YearsCodePro'].replace('NA',0)
df['YearsCodePro'].unique()

array([0, '9', '23', '7', '4', '21', '3', '15', 1, '10', '2', '6', '14',
       '5', '19', '13', '16', '28', '1', '30', '11', '8', '25', '32',
       '24', '40', '17', '45', '29', '12', '31', '20', '18', '50', '27',
       '43', '22', '26', '38', '33', '44', '35', '34', '37', '42', '41',
       51, '47', '36', '39', '48', '46', '49'], dtype=object)

In [ ]:
df['WorkExp'].unique()
df['WorkExp'] = df['WorkExp'].replace('NA',0)
df['WorkExp'].unique()

array([ 0., 10., 23.,  7.,  6., 22.,  4.,  5.,  3.,  9., 15., 14., 18.,
       13., 39.,  2., 28., 12., 17., 25., 11., 16., 30., 20., 24., 19.,
        1.,  8., 26., 38., 27., 32., 21., 40., 34., 35., 29., 33., 37.,
       36., 31., 48., 45., 42., 43., 44., 50., 41., 46., 47., 49.])

In [ ]:
# Scale numerical variables
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [ ]:
######## NO ###############
# Calculate cosine similarity between professionals
similarity_matrix = cosine_similarity(df)

In [ ]:
# Step 4: Model Training (User-based Collaborative Filtering)
# Split data into train and test sets
train_data, test_data = train_test_split(df, test_size=0.2)

#minkowski is good so far

# Fit nearest neighbors model
knn_model = NearestNeighbors(n_neighbors=5, metric='minkowski')
knn_model.fit(train_data.values)

NearestNeighbors()

In [ ]:
questions = {
    "MainBranch": "What describes you the best?",
    "Employment": "Which of the following best describes your current employment status?",
    "RemoteWork": "Which best describes your current work situation?",
    "CodingActivities": "Which of the following best describes the code you write outside of work?",
    "EdLevel": "Which of the following best describes the highest level of formal education that you've completed?",
    "LearnCode": "How do you learn to code?",
    "LearnCodeOnline": "What online resources do you use to learn to code?",
    "LearnCodeCoursesCert": "What online courses or certifications do you use to learn to code? ",
    "YearsCode": "Including any education, how many years have you been coding in total?",
    "YearsCodePro": "NOT including education, how many years have you coded professionally (as a part of your work)?",
    "DevType": "Which of the following describes your current job, the one you do most of the time?",
    "Country": "Where do you live?",
    "LanguageHaveWorkedWith": "What languages have you worked with?",
    "LanguageWantToWorkWith": "What languages are you looking to work with?",
    "DatabaseHaveWorkedWith": "What databases have you worked with?",
    "DatabaseWantToWorkWith": "What databases are you looking to work with?",
    "PlatformHaveWorkedWith": "What platform have you worked on?",
    "PlatformWantToWorkWith": "What platform are you looking to work on?",
    "WebframeHaveWorkedWith": "What Webframeworks have you worked with?",
    "WebframeWantToWorkWith": "What Webframeworks do you want to work with?",
    "MiscTechHaveWorkedWith": "What other Miscellaneous Tech have you worked with?",
    "MiscTechWantToWorkWith": "What other Miscellaneous Tech do you want to work with?",
    "ToolsTechHaveWorkedWith": "Any other Tools and Technology familiar with?",
    "ToolsTechWantToWorkWith": "Any other Tools and Technology you want to get familiar with?",
    "NEWCollabToolsHaveWorkedWith": "What collaboration tools have you used?",
    "NEWCollabToolsWantToWorkWith": "Which other collaboration tools do you wish to use?",
    "ICorPM": "Are you an individual contributor or people manager?",
    "WorkExp": "How many years of working experience do you have?",
    "ProfessionalTech": "Which other professional technologies do you use?",
    "Industry": "What industry is the company you work for in?"
}

In [ ]:
questions.values()

dict_values(['What describes you the best?', 'Which of the following best describes your current employment status?', 'Which best describes your current work situation?', 'Which of the following best describes the code you write outside of work?', "Which of the following best describes the highest level of formal education that you've completed?", 'How do you learn to code?', 'What online resources do you use to learn to code?', 'What online courses or certifications do you use to learn to code? ', 'Including any education, how many years have you been coding in total?', 'NOT including education, how many years have you coded professionally (as a part of your work)?', 'Which of the following describes your current job, the one you do most of the time?', 'Where do you live?', 'What languages have you worked with?', 'What languages are you looking to work with?', 'What databases have you worked with?', 'What databases are you looking to work with?', 'What platform have you worked on?', '

In [ ]:
row = originalDF.loc[22023]
for column, value in row.items():
    print(f"{questions[column]}=> {value}\\")

What describes you the best?=> I am a developer by profession\
Which of the following best describes your current employment status?=> Employed, full-time\
Which best describes your current work situation?=> Remote\
Which of the following best describes the code you write outside of work?=> I don’t code outside of work\
Which of the following best describes the highest level of formal education that you've completed?=> Associate degree (A.A., A.S., etc.)\
How do you learn to code?=> Books / Physical media;Other online resources (e.g., videos, blogs, forum)\
What online resources do you use to learn to code?=> Formal documentation provided by the owner of the tech;Online challenges (e.g., daily or weekly coding challenges);Stack Overflow\
What online courses or certifications do you use to learn to code? => nan\
Including any education, how many years have you been coding in total?=> 16\
NOT including education, how many years have you coded professionally (as a part of your work)?=> 10

In [ ]:
######## NO ###############
train_data.head()
train_data.loc[82395]
# Select the row at index 2.
row = train_data.loc[82395]

# Print the columns where the value is 1.
print(row.index[row == 1])

Index(['EdLevel_Bachelor’s degree (B.A., B.S., B.Eng., etc.)',
       'Industry_Information Services, IT, Software Development, or other Technology',
       'Country_United States of America', 'RemoteWork_Remote',
       'DevType_Developer, full-stack'],
      dtype='object')


In [ ]:
print(originalDF.loc[82395])

MainBranch                                                                                                                                                                                                                                                            I am a developer by profession
Employment                                                                                                                                                                                                                                                                       Employed, full-time
RemoteWork                                                                                                                                                                                                                                                                                    Remote
CodingActivities                                                                                                         

In [ ]:
random_index = np.random.randint(0, len(test_data))  # Generate a random index
random_professional = test_data.iloc[random_index]

In [ ]:
print(random_index)

5016


In [ ]:
random_professional.shape

(871,)

In [ ]:
# Convert the attributes to a numpy array and reshape it
new_professional_array = np.array(random_professional).reshape(1, -1)

In [ ]:
# Find nearest neighbors to the new professional
distances, indices = knn_model.kneighbors(new_professional_array)

# Get recommended mentors and their distances
recommended_mentors = train_data.iloc[indices[0]]
neighbor_distances = distances[0]


# Calculate confidence scores based on distances
# You may need to normalize distances if required
# For example, you can use the inverse of distances as confidence scores
# Or you can use a normalization method such as min-max scaling or z-score normalization
epsilon = 1e-10
confidence_scores = 1 / (neighbor_distances  + epsilon) # Example of using inverse of distances as confidence scores


# Normalize confidence scores to percentages
confidence_scores_percent = (confidence_scores - confidence_scores.min()) / (confidence_scores.max() - confidence_scores.min()) * 100

# Print recommended mentors alongside their confidence scores
for i, (index, mentor) in enumerate(recommended_mentors.iterrows()):
    print(f"Mentor {i+1}:")
    print(f"Confidence Score: {confidence_scores_percent[i]:.2f}%")  # Print confidence score as percentage
    print()

Mentor 1:
Confidence Score: 100.00%

Mentor 2:
Confidence Score: 74.68%

Mentor 3:
Confidence Score: 55.22%

Mentor 4:
Confidence Score: 22.25%

Mentor 5:
Confidence Score: 0.00%



In [ ]:
recommended_mentors

,YearsCode,YearsCodePro,WorkExp,CodingActivities_Bootstrapping a business,CodingActivities_Contribute to open-source projects,CodingActivities_Freelance/contract work,CodingActivities_Hobby,CodingActivities_I don’t code outside of work,CodingActivities_NA,CodingActivities_Other (please specify):,...,DevType_NA,DevType_Other (please specify):,DevType_Product manager,DevType_Project manager,DevType_Research & Development role,DevType_Scientist,DevType_Security professional,"DevType_Senior Executive (C-Suite, VP, etc.)",DevType_Student,DevType_System administrator
21184,0.122129,0.711457,-0.654337,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
5710,0.122129,0.818517,-0.654337,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
87408,0.027908,0.604396,-0.654337,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
75952,0.310570,0.711457,-0.654337,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
40495,0.122129,0.497335,-0.654337,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
random_professional

In [ ]:
# Get unique values for each column
unique_values_per_column = {}

for column in df.columns:
    unique_values_per_column[column] = df[column].unique()

# Print unique values for each column
for column, unique_values in unique_values_per_column.items():
    print(f"Unique values for column '{column}':")
    print(unique_values)
    print()

In [ ]:
# Retrieve the label encoder for the specific column
label_encoder = label_encoders["CodingActivities"]

# Use inverse_transform to decode the encoded values
original_values = label_encoder.inverse_transform(np.array([89]))

In [ ]:
original_values

array(['Hobby;Freelance/contract work'], dtype=object)

In [ ]:
import pickle

# # Save the encoder.
# with open('/content/drive/MyDrive/Eduforge/model.pkl', 'wb') as f:
#     pickle.dump(knn_model, f)

# Save the encoder.
# with open('/content/drive/MyDrive/Eduforge/scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

with open('/content/drive/MyDrive/Eduforge/train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)

# # Save the encoder.
# with open('encoder.pkl', 'wb') as f:
#     pickle.dump(encoder, f)

# # Save the scaler.
# with open('scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

In [ ]:
from sklearn.metrics import precision_score, recall_score, average_precision_score

# Assuming 'ground_truth' contains the ground truth labels and 'predictions' contains the recommendations
# For precision and recall, you may need to convert predictions to binary format (e.g., 1 for recommended, 0 for not recommended)

precision = precision_score(ground_truth, predictions)
recall = recall_score(ground_truth, predictions)
map_score = average_precision_score(ground_truth, predictions)

print("Precision:", precision)
print("Recall:", recall)
print("Mean Average Precision (MAP):", map_score)

In [ ]:
unique_values = {}
for column in df.columns:
  unique_values[column] = [str(value) for value in df[column].unique()]


unique_values_all_columns = set()
for column, values in unique_values.items():
  new_values = [item for sublist in values for item in sublist.split(";")]
  unique_values_all_columns.update(new_values)

len(unique_values_all_columns)

679

In [ ]:
df.shape

(89184, 30)

In [ ]:
# Use a different set of stop words
stop_words = list(unique_values_all_columns)

# Modify your tokenizer to keep stop words
tokenizer = lambda x: x.split(";")

# Instantiating the Vectorizer
vectorizer = TfidfVectorizer(vocabulary=stop_words, tokenizer=tokenizer)

# Fit the vectorizer to your data
x = vectorizer.fit_transform(df)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [ ]:
# Fill missing values with the most frequent value of each column
df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))
df = df.applymap(str)

df_transformed = df.apply(vectorizer.fit_transform)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [ ]:
print(df_transformed.to_numpy().shape)

(30,)


In [ ]:
df[numerical_columns].fillna(0, inplace=True)
df[categorical_columns].fillna("NA", inplace=True)

<ipython-input-24-f8c925b10467>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numerical_columns].fillna(0, inplace=True)
<ipython-input-24-f8c925b10467>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical_columns].fillna("NA", inplace=True)


In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from summa import summarizer
from textblob import TextBlob
from collections import Counter

def analyze_feedback(feedback):
    # Sentiment Analysis
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(feedback)

    # Topic Modeling
    summarized_feedback = summarizer.summarize(feedback)

    # Named Entity Recognition
    feedback_blob = TextBlob(feedback)
    named_entities = feedback_blob.noun_phrases

    # Actionable Insights
    # Counting the most common noun phrases
    noun_phrase_counts = Counter(named_entities)
    top_noun_phrases = noun_phrase_counts.most_common(3)

    # Actionable Suggestions based on sentiment
    actionable_suggestions = []
    if sentiment_score['compound'] < -0.2:
        actionable_suggestions.append("Address negative aspects mentioned in the feedback.")
    elif sentiment_score['compound'] > 0.2:
        actionable_suggestions.append("Leverage positive aspects mentioned in the feedback.")

    return {
        "sentiment_score": sentiment_score,
        "summarized_feedback": summarized_feedback,
        "top_noun_phrases": top_noun_phrases,
        "actionable_suggestions": actionable_suggestions
    }

# Example feedback text
feedback_text = """
Thank you for sharing your background and preferences with us. Based on your responses, it's clear that you have a solid foundation in coding, with a focus on backend development. Your interest in languages like C++, Lua, Python, and Rust suggests a strong inclination towards systems programming and backend technologies.

Given your experience with platforms like Heroku and frameworks like FastAPI and React, you seem well-versed in building scalable web applications. Additionally, your familiarity with tools like APT, Cargo, and GNU GCC indicates a proficiency in software development workflows.

Considering your employment status as a student and part-time employee, along with your remote work situation, it's evident that flexibility and efficiency are crucial for you. Hence, I would suggest exploring opportunities to enhance your collaboration skills, perhaps by leveraging tools like Visual Studio Code more effectively for remote teamwork.

Keep up the good work, and don't hesitate to reach out if you need further guidance or assistance.

"""

# Analyzing feedback
analysis_result = analyze_feedback(feedback_text)

# Printing the analysis result
print("Sentiment Score:", analysis_result["sentiment_score"])
print("Summarized Feedback:", analysis_result["summarized_feedback"])
print("Top Noun Phrases:", analysis_result["top_noun_phrases"])
print("Actionable Suggestions:", analysis_result["actionable_suggestions"])


Sentiment Score: {'neg': 0.006, 'neu': 0.748, 'pos': 0.246, 'compound': 0.9896}
Summarized Feedback: Hence, I would suggest exploring opportunities to enhance your collaboration skills, perhaps by leveraging tools like Visual Studio Code more effectively for remote teamwork.
Top Noun Phrases: [('thank', 1), ('based', 1), ('solid foundation', 1)]
Actionable Suggestions: ['Leverage positive aspects mentioned in the feedback.']


In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

def extract_actionables(feedback):
    # Process the feedback text with spaCy
    doc = nlp(feedback)

    # Extract actionable verbs and phrases
    actionables = []
    for token in doc:
        if token.pos_ == "VERB" and token.dep_ in ["ROOT", "xcomp"]:
            actionables.append(token.text)
        elif token.pos_ == "NOUN" and token.dep_ == "dobj":
            actionables.append(token.text)
        elif token.pos_ == "VERB" and token.dep_ == "acl":
            actionables.append(token.text)
        elif token.pos_ == "VERB" and token.dep_ == "advcl":
            actionables.append(token.text)
        elif token.pos_ == "AUX" and token.dep_ == "aux":
            actionables.append(token.text)

    return actionables

def generate_actionable_suggestions(actionables):
    actionable_suggestions = []

    if actionables:
        actionable_suggestions.append("Consider taking the following actions based on the feedback:")
        actionable_suggestions.extend("- {}".format(action) for action in actionables)
    else:
        actionable_suggestions.append("No specific actionable suggestions identified in the feedback.")

    return actionable_suggestions

# Example usage
feedback_text = """
I really enjoyed the service provided by your team. However, I think there is room for improvement in the delivery time. The product quality is excellent, but sometimes the packaging is not secure enough. Overall, I had a good experience but would appreciate faster delivery and better packaging.
"""

# Extract actionables from feedback
actionables = extract_actionables(feedback_text)

# Generate actionable suggestions
actionable_suggestions = generate_actionable_suggestions(actionables)

# Print actionable suggestions
print("Actionable Suggestions extracted from the feedback:")
for suggestion in actionable_suggestions:
    print(suggestion)


Actionable Suggestions extracted from the feedback:
Consider taking the following actions based on the feedback:
- enjoyed
- service
- provided
- think
- had
- experience
- would
- delivery


In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
!pip install summa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54386 sha256=7bc9b16c888619867b46db0e289d408f84fe9c8254aac1dd78490fe0bada2cbd
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa
